# Statistical Testing

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt 
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats as sm_stats
import statsmodels.stats.api as sms

# 1. Data samples: two groups, each group has 10 students
- Those who went to lectures (attended at least 50% of the lectures)
- Those who did not attend lectures (others)

In [ ]:
np.random.seed(123)
sample_size = 10
m1 = 80
m2 = 78
s = 5

attended = stats.norm(loc=m1, scale=s)
not_attended = stats.norm(loc=m2, scale=s)

# rvs: Random Varieties
a_sample = attended.rvs(size=sample_size)
na_sample = not_attended.rvs(size=sample_size)

## 1.2 What does our data look like?

In [ ]:
a_sample

In [ ]:
na_sample

In [ ]:
a_sample.mean()

In [ ]:
na_sample.mean()

Students who *didn't* go to lectures have a higher final grade on average.

#### Theoretically, we should observe a difference

In [ ]:
# pdf: Probability Density Function
x = np.linspace(60, 100, 100)
plt.plot(x, attended.pdf(x), 'b')
plt.plot(x, not_attended.pdf(x), 'r')

#### So can I declare that not going to lectures improves the final grade in the subject?

In [ ]:
df = pd.DataFrame({'attended': np.repeat([True, False], 10), 'score': np.concatenate((a_sample, na_sample))})
sns.boxplot(x=df['attended'], y=df['score'])

#### The basic problem is that we are not measuring whole populations, but only their samples

So we only work with estimates. **How ​​can we be sure of them?**

<img src="https://s3-eu-west-1.amazonaws.com/blog.omniconvert.com-media/blog/wp-content/uploads/2019/10/21150245/sample-size-definition.png " width="50%" />

#### Statistics offers us tools to find out if there are differences in two statistical sets
- Is there any difference at all? There will probably be some
- Is the difference small or large? Statistics will not help us much here
- Does the measured difference have any practical value? Statistics do not give us an answer to this either
- **Is the measured difference real or just due to chance? Statistics can help us here!**

## 1.3 So how to verify that the difference between the groups is real and not given by chance?
- Statistical hypotheses testing
- When testing hypotheses, we consider the probability that we could have achieved the given result if the experimental procedure had no effect
- Assumption of null effect (difference) = **null hypothesis** = $H_0$

**$H_0$ = The average final grade of students who attended lectures is the same as those who did not attend lectures.**

- Alternative hypothesis $H_1$ (if the null $H_0$ would not apply)

**$H_1$ = The average final grade of students who attended lectures is different/greater/smaller than those who did not attend lectures.**

**Error of the first kind (Type I) and error of the second kind (Type II)**

<img src="https://chemicalstatistician.files.wordpress.com/2014/05/pregnant.jpg" width="35%" />

### 1.3.1 Basic steps of hypothesis testing
1. Determine the null hypothesis $H_0$ and alternate hypothesis $H_1$
2. Set a significance level 𝛼 
3. Compute p-value using suitable test statistic 𝑇  
4. Make a decision based on p-value and 𝛼 

**We just need to choose a suitable statistical test!**

<!-- <img src="https://i.stack.imgur.com/idDTA.png" /> //-->
[<img src="img/critical-p-values.png" />](https://www.geo.fu-berlin.de/en/v/soga/Basics-of-statistics/Hypothesis-Tests/Introduction-to-Hypothesis-Testing/Critical-Value-and-the-p-Value-Approach/index.html)

### 1.3.2 Significance tests: Student's t-test

- the t-statistic was introduced in 1908 by William Sealy Gosset while he was working as a chemist at the Guinness brewery.
- t-test is based on **t-distribution**.
- the t-distribution is similar to the normal distribution, but has more mass in the tails. As the number of observations increases, it approaches a normal distribution.
- the t-test for two paired samples compares whether the difference of pairs of observations is different from zero.
 
Calculation of t-statistics (for two independent samples):

$t = \frac{\overline{X_1} - \overline{X_2}}{s_p \sqrt{\frac{1}{n_1} - \frac{1}{n_2}}}$ 

$s_p = \sqrt{\frac{(n_1 - 1) s^2_{X_1} + (n_2 - 1) s^2_{X_2}}{n_1 + n_2 - 2}}$

Assumptions of the t-test
- The input values are from a normal distribution
- values come from distributions with similarly large variance (dispersion of values) - there is a t-test correction for distributions with different variance (*Welch's t-test*).
- *t-test is resistant to slight deviations from these assumptions.*

In [ ]:
# stats.ttest_ind(a_sample, na_sample)
stat, p = stats.ttest_ind(a_sample, na_sample)
print('Ttest_indResult: statistic=', stat, 'pvalue=', p)

#### Interpretation

In [ ]:
alpha = 0.05
if p > alpha:
    print('Same distributions (fail to reject H0)')
else:
    print('Different distributions (reject H0)')


*t-test*: $H_0$ = the means of two populations are equal

- Assumption of null effect = **null hypothesis** = $H_0$

in the domain: **$H_0$ = The average final grade of students who attended lectures is the same as those who did not attend lectures.**

*We cannot reject $H_0$ (fail to reject $H_0$) based on the result of a *t-test* with two samples (there are 10 students in each)*

**We continue with the tests!!!**

### 1.3.3 Normality assumption testing

1. Visual inspection using a histogram or so-called quantile-quantile graph (QQ graph) - especially for large samples
2. Normality test, e.g. using the **Shapiro-Wilk** test.

In [ ]:
# sns.distplot(a_sample, bins=5)
sns.histplot(a_sample, bins=5)

In [ ]:
# sns.distplot(na_sample, bins=5)
sns.histplot(na_sample, bins=5)

#### 1.3.4.1 QQ plot

A quantile-quantile plot (*QQ-plot*) is a visual method used to determine whether two data sets come from the same distribution. Most commonly, it compares the distribution of a sample with a theoretical normal distribution. Each point on the plot represents the quantile value in the first and second data sets being compared.

In [ ]:
_ = sm.ProbPlot(a_sample, fit=True).qqplot(line='45')

In [ ]:
_ = sm.ProbPlot(na_sample, fit=True).qqplot(line='45')

**How ​​to interpret a QQ plot?**

<img src="https://i.stack.imgur.com/ZXRkL.png" />

Source: https://stats.stackexchange.com/questions/101274/how-to-interpret-a-qq-plot

#### 1.3.5 Shapiro-Wilk normality test

- The Shapiro-Wilk test tests the null hypothesis that the data come from a normal distribution.
- If $p < 0.05$, we reject the null hypothesis and the data probably come from a non-normal distribution. If $p > 0.05$, we do not reject the null hypothesis, that is, based on the data, we cannot declare that the data come from a different than normal distribution.
- `scipy.stats.shapiro`: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html

In [ ]:
stats.shapiro(a_sample)

In [ ]:
stats.shapiro(na_sample)

### 1.3.4 Variance test: Levene Test

- Levene's test tests the null hypothesis that all input samples come from distributions with equal variances.
- If we do not reject the null hypothesis ($p > 0.05$), it means that, based on the data, we cannot claim that the samples come from distributions with different variances.

**However, we did not prove that the averages are the same.**

**How ​​is that possible?**

We generated data from distributions with different means.

**Type II error** - we used an underpowered test.

## 1.4 Statistical power = $1 - \beta$

- The probability of finding a significant difference if one exists (rejecting $H_0$ when it is false).
- With low test power, we cannot identify smaller effects (differences).
- We can increase the power by increasing the number of participants (observations).

**The power of the test, or the required number of participants/observations, can be calculated in advance!**

- To do this, we need to estimate the *effect size*.
- There are several ways, such as **Cohen's d**.

$d = \frac{\overline{x_1} - \overline{x_2}}{s}$

where

$s = \sqrt{\frac{(n_1 - 1) s^2_{X_1} + (n_2 - 1) s^2_{X_2}}{n_1 + n_2 - 2}}$

- small effect = 0.2, medium effect = 0.5, large effect = 0.8

**In our example there is a difference, but we were not able to measure it: We only had 10 observations !!!**

In [ ]:
def cohen_d(x1, x2):
    nx1 = len(x1)
    nx2 = len(x2)
    s = np.sqrt(((nx1-1) * np.std(x1, ddof=1)**2 + (nx2-1) * np.std(x2, ddof=1)**2) / (nx1 + nx2 - 2))
    return (np.abs(np.mean(x1) - np.mean(x2))) / s

In [ ]:
c_d = cohen_d(a_sample, na_sample)
c_d

In [ ]:
sm_stats.power.tt_ind_solve_power(c_d, len(a_sample), 0.05, None, 1)

`statsmodels.stats.power.tt_ind_solve_power`:
https://www.statsmodels.org/stable/generated/statsmodels.stats.power.tt_ind_solve_power.html

## 1.5 Effect size: Large effect (0.8)

In [ ]:
sm_stats.power.tt_ind_solve_power(c_d, None, 0.05, 0.8, 1)

In fact, we would need fewer observations (since we generated the data, we know the true values ​​of the means and standard deviations):

In [ ]:
sm_stats.power.tt_ind_solve_power((m1-m2)/s, None, 0.05, 0.8, 1)

# 2. Bigger data samples of 100 students for each group

In [ ]:
a_sample2 = attended.rvs(100)
na_sample2 = not_attended.rvs(100)

In [ ]:
a_sample2.mean()

In [ ]:
na_sample2.mean()

In [ ]:
plt.rcParams["figure.figsize"] = (10,7)

In [ ]:
df2 = pd.DataFrame({'attended': np.repeat([True, False], 100), 'score': np.concatenate((a_sample2, na_sample2))})
sns.boxplot(x=df['attended'], y=df['score'])

In [ ]:
stats.ttest_ind(a_sample2, na_sample2)

- **Based on the t-test, we reject the null hypothesis**
- But the average estimates still do not correspond to the actual values.
- *How to find out the accuracy of this estimate, or what is the true value of the mean?*

## 2.2 Confidence interval

The confidence level $C$ (e.g., 95%) tells us the percentage of confidence intervals that would contain the true estimated population value (e.g., the mean) if we were to test infinitely many samples from the population.

When the population's standard deviation is unknown, we use values from the t-distribution:

$ \overline{x} \pm t_{\alpha}(n-1)\frac{s}{\sqrt{n}} $

$ \alpha = \frac{1-C}{2} $

**Example: public opinion poll**

<img src="img/election-poll2.png" alt="Confidence intervals in an election poll" width="20%"/>

In [ ]:
sms.DescrStatsW(a_sample).tconfint_mean()

In [ ]:
sms.DescrStatsW(na_sample).tconfint_mean()

In [ ]:
plt.rcParams["figure.figsize"] = (6,4)

In [ ]:
sns.barplot(x='attended', y='score', data=df, capsize=0.1, errwidth=2, palette=sns.color_palette("Blues"))

In [ ]:
sms.DescrStatsW(a_sample2).tconfint_mean()

In [ ]:
sms.DescrStatsW(na_sample2).tconfint_mean()

In [ ]:
sns.barplot(x='attended', y='score', data=df2, capsize=0.1, errwidth=2, palette=sns.color_palette("Blues"))

## 2.3 We had two groups—what if we had more?

$\alpha = 0.05$

* 1 test: 5% probability of error
* 2 tests: $1 – (1-0.05)^2 \approx 9.75\%$ probability of error
* 10 tests: $1 – (1-0.05)^{10} \approx 40.1\%$ probability of error
* 25 tests: $1 – (1-0.05)^{25} \approx 72.3\%$ probability of error

As the number of tested groups increases, the Type I error rate also increases.

#### We can control Type I errors

- **FWER (Familywise Error Rate)** = The probability of rejecting at least one true $H_i$ (*making a Type I error*) when testing a group of null hypotheses.
- **Bonferroni correction**

$p_i \leq \alpha/m$, where $m$ is the number of hypotheses (pairwise tests).

For 4 test conditions, $\alpha = 0.05$ leads to 6 pairwise tests, so $p_i \leq 0.05/6 = 0.0083$.

*Or even better—use a test for multiple groups (e.g., ANOVA) combined with pairwise post-hoc tests.*

#### Reporting the p-value is not enough
- It only indicates whether there is any effect
- It depends on the sample size; with sufficiently large samples, you will almost always find a significant difference
- You should also report the **effect size**. http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3444174


# 3. BIGGER data samples: 100 000 students for each group

In [ ]:
attended2 = stats.norm(80, 5)
not_attended2 = stats.norm(79.9, 5)

a_sample3 = attended2.rvs(100000)
na_sample3 = not_attended2.rvs(100000)

In [ ]:
a_sample3.mean()

In [ ]:
na_sample3.mean()

In [ ]:
# stats.ttest_ind(a_sample3, na_sample3)
stat, p = stats.ttest_ind(a_sample3, na_sample3)
print('Ttest_indResult: statistic=', stat, 'pvalue=', p)

#### Interpretation

In [ ]:
alpha = 0.05
if p > alpha:
    print('Same distributions (fail to reject H0)')
else:
    print('Different distributions (reject H0)')


in the domain: **$H_0$ = The average final grade of students who attended lectures is the same as those who did not attend lectures.**

**We reject $H_0$ (Reject $H_0$) based on the result of the t-test with larger two samples !!!**

#### *t-test* one more time

- $H_0$ = the means of two populations are equal
- Fail to Reject $H_0$: No difference between the sample means
- Reject $H_0$: Some difference between the sample means

## Finally, statistical hypothesis testing WORKS correctly !!!

# You can already use statistics to test hypotheses :)

**Online courses**

- Statistical Inference (https://www.coursera.org/learn/statistical-inference; part of the *Data Science* specialization)
- Statistics with Python Specialization (https://www.coursera.org/specializations/statistics-with-python)
- Introduction to Statistics as Covered in the Social, Behavioral, and Natural Sciences (https://www.udemy.com/course/introduction-to-statistics/)
- Statistics for Business Analytics and Data Science A-Z (https://www.udemy.com/course/data-statistics/)
- Statistics (Khan Academy): https://www.youtube.com/playlist?list=PL1328115D3D8A2566

**References**
- Brian Caffo: Little Inference Book (https://leanpub.com/LittleInferenceBook)
- Alex Reinhard: Statistics Done Woefully Wrong (https://www.statisticsdonewrong.com/)
- Will Kurt: Bayesian Statistics the Fun Way (https://nostarch.com/learnbayes)
- https://github.com/FIIT-IAU

<!--
Next step: Bayesian inference
- Bayesian inference is a method of statistical inference in which Bayes' theorem is used to update the probability for a hypothesis as more evidence or information becomes available.
//-->